In [3]:
import glob
from tqdm import tqdm
from src.create_annotations import *
from PIL import Image
import os

category_ids = {
    # "Background":0,
    "General trash":1,
    "Paper":2,
    "Paper pack":3,
	"Metal":4,
	"Glass":5,
	"Plastic":6,
	"Styrofoam":7,
	"Plastic bag":8,
	"Battery":9,
	"Clothing":10
    }

category_colors = {
    # "(0, 0, 0)":0,
    "(192, 0, 128)":1,
    "(0, 128, 192)":2,
    "(0, 128, 64)":3,
    "(128, 0, 0)":4,
    "(64, 0, 128)":5,
    "(64, 0, 192)":6,
    "(192, 128, 64)":7,
    "(192, 192, 128)":8,
    "(64, 64, 128)":9,
    "(128, 0, 192)":10
}

In [37]:
# Get the standard COCO JSON format
coco_format = get_coco_json_format()

cp_data_dir = "/opt/ml/input/data/fold/fold0/output"
mask_path = os.path.join(cp_data_dir, 'SegmentationClass/')

# Create category section
coco_format["categories"] = create_category_annotation(category_ids)

# Create images and annotations sections
coco_format["images"], coco_format["annotations"], annotation_cnt, multipolygon, invalid_pixel = images_annotations_info(mask_path)

# This id will be automatically increased as we go
annotation_id = 0
image_id = 0
annotations = []
images = []
multipolygon = []
invalid_pixel = []

for mask_image in tqdm(glob.glob(mask_path + "*.png")):
    # The mask image is *.png but the original image is *.jpg.
    # We make a reference to the original file in the COCO JSON file
    original_file_name = os.path.basename(mask_image).split(".")[0] + ".jpg"

    mask_image = Image.open()

    # Open the image and (to be sure) we convert it to RGB
    mask_image_open = Image.open(mask_image).convert("RGB")
    w, h = mask_image_open.size
    
    # "images" info 
    image = create_image_annotation(original_file_name, w, h, image_id)
    images.append(image)
    sub_masks = create_sub_masks(mask_image_open, w, h)
    
    for color, sub_mask in sub_masks.items():

        if color not in category_colors.keys():
            continue

        category_id = category_colors[color]

        contours = measure.find_contours(np.array(sub_mask), 0.5, positive_orientation = 'low')
        polygons = []
        segmentations = []

        for contour in contours:
            for i in range(len(contour)):
                row, col = contour[i]
                contour[i] = (col - 1, row - 1)
            poly = Polygon(contour)
            poly.simplify(1.0, preserve_topology = False)

            if poly.is_empty:
                continue

            polygons.append(poly)
            segmentation = np.array(poly.exterior.coords, dtype = np.int).ravel().tolist()
            segmentations.append(segmentation)

            min_x, min_y, max_x, max_y = poly.bounds
            width = max_x - min_x
            height = max_y - min_y
            bbox = (min_x, min_y, width, height)
            area = poly.area

            annotation = {
                "id": annotation_id,
                "image_id": image_id,
                "category_id": int(category_id),
                "segmentation": [segmentation],
                "area": area,
                "iscrowd": 0,
                "bbox": bbox
            }
            annotation_id += 1
            annotations.append(annotation)

    image_id += 1

coco_format["images"], coco_format["annotations"], annotation_cnt, multipolygon, invalid_pixel = images, annotations, annotation_id, multipolygon, invalid_pixel

with open(os.path.join(cp_data_dir, "output.json"),"w") as outfile:
    json.dump(coco_format, outfile, indent = 4)

print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))

  6%|▌         | 199/3271 [02:56<45:20,  1.13it/s]


Created 12811 annotations for images in folder: /opt/ml/input/data/copy_paste/output/SegmentationClass/
